In [9]:
import fastf1
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import importlib.util

fastf1.Cache.enable_cache('../data/raw')  # Crear directorio cache para mejorar rendimiento

In [10]:
# Importar las funciones de recolección de datos usando importlib
data_collection_path = os.path.join(os.path.dirname(os.getcwd()), 'src', 'data_collection.py')
spec = importlib.util.spec_from_file_location("data_collection", data_collection_path)
data_collection = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_collection)

# Acceder a las funciones del módulo
get_session_data = data_collection.get_session_data

In [11]:
class F1PerformanceCalculator:
    def __init__(self):
        """Initialize the F1 Performance Calculator"""
        # Enable FastF1 cache for better performance
        pass
    
    def validate_round(self, year, round_num):
        """
        Validate if a round exists in the given year
        Returns tuple (is_valid, max_rounds, error_message)
        """
        try:
            schedule = fastf1.get_event_schedule(year)
            max_rounds = len(schedule)
            
            if round_num < 1:
                return False, max_rounds, f"La ronda debe ser mayor a 0. Ronda solicitada: {round_num}"
            elif round_num > max_rounds:
                return False, max_rounds, f"Ronda {round_num} no existe en {year}. Máximo: {max_rounds} rondas"
            else:
                return True, max_rounds, None
                
        except Exception as e:
            return False, 0, f"Error al obtener el calendario de {year}: {e}"
        
    def is_wet_race(self, session):
        """
        Determine if a race was wet based on compound usage and weather data
        Returns True if wet, False if dry
        """
        try:
            # Get race results
            results = session.results
            laps = session.laps
            
            # Method 1: Check if intermediate or wet compounds were used significantly
            if not laps.empty:
                compound_counts = laps['Compound'].value_counts()
                wet_compounds = ['INTERMEDIATE', 'WET']
                wet_laps = sum(compound_counts.get(compound, 0) for compound in wet_compounds)
                total_laps = len(laps)
                
                # If more than 20% of laps were on wet compounds, consider it wet
                if total_laps > 0 and (wet_laps / total_laps) > 0.2:
                    return True
            
            # Method 2: Check weather data if available
            try:
                weather = session.weather_data
                if not weather.empty:
                    # If significant rainfall during the race
                    avg_rainfall = weather['Rainfall'].mean()
                    if avg_rainfall > 0.1:  # Threshold for wet conditions
                        return True
            except:
                pass
                
            return False
            
        except Exception as e:
            print(f"Error determining race conditions: {e}")
            return False
    
    def calculate_race_performance_score(self, driver_results):
        """
        Calculate performance score based on grid position vs finish position
        Higher score = better performance
        """
        try:
            if driver_results.empty:
                return 0.0
            
            # Get the driver's result
            grid_pos = driver_results['GridPosition'].iloc[0]
            finish_pos = driver_results['Position'].iloc[0]
            
            # Handle DNF cases
            if pd.isna(finish_pos) or finish_pos == 0:
                finish_pos = 21  # Assign last position for DNF
            
            # Handle grid penalties or back-of-grid starts
            if pd.isna(grid_pos) or grid_pos == 0:
                grid_pos = 20
            
            # Calculate performance score
            # Positive score = finished better than started
            # Negative score = finished worse than started
            position_change = grid_pos - finish_pos
            
            # Normalize based on grid position (easier to gain from back)
            if grid_pos <= 10:
                # Front runners: harder to improve, penalize drops more
                score = position_change * 1.2
            else:
                # Back runners: easier to improve, reward gains more
                score = position_change * 0.8
            
            # Bonus for points finish
            if finish_pos <= 10:
                score += 2
            
            # Penalty for DNF
            if finish_pos > 20:
                score -= 5
            
            return score
            
        except Exception as e:
            print(f"Error calculating performance score: {e}")
            return 0.0
    
    def get_previous_races(self, target_year, target_round, num_dry=7, num_wet=5):
        """
        Get previous races before the target race, separating dry and wet races
        """
        dry_races = []
        wet_races = []
        
        # Start from the previous race in the same season
        current_year = target_year
        current_round = target_round - 1
        limit_year = 2024
        
        # Go back through races until we have enough of each type
        while (len(dry_races) < num_dry or len(wet_races) < num_wet) and current_year >= limit_year:
            try:
                if current_round <= 0:
                    current_year -= 1
                    if current_year < limit_year:
                        break
                    # Get the schedule for the previous year and validate
                    try:
                        schedule = fastf1.get_event_schedule(current_year)
                        current_round = len(schedule)
                        print(f"Moving to {current_year}, max rounds: {current_round}")
                    except Exception as e:
                        print(f"Error getting schedule for {current_year}: {e}")
                        current_year -= 1
                        continue
                
                # Validate round before attempting to load
                is_valid, max_rounds, error_msg = self.validate_round(current_year, current_round)
                if not is_valid:
                    print(f"Skipping {current_year} Round {current_round}: {error_msg}")
                    current_round -= 1
                    continue
                
                print(f"Loading {current_year} Round {current_round}...")
                
                # Verificar que get_session_data no devuelva None
                try:
                    session = get_session_data(current_year, current_round, 'Race')
                    if session is None:
                        print(f"  -> Session is None for {current_year} Round {current_round}")
                        current_round -= 1
                        continue
                except Exception as e:
                    print(f"  -> Error getting session data: {e}")
                    current_round -= 1
                    continue
                
                # Verificar que session.results no sea None
                if not hasattr(session, 'results') or session.results is None:
                    print(f"  -> No results available for {current_year} Round {current_round}")
                    current_round -= 1
                    continue
                    
                if session.results.empty:
                    print(f"  -> Empty results for {current_year} Round {current_round}")
                    current_round -= 1
                    continue
                
                is_wet = self.is_wet_race(session)
                
                race_data = {
                    'year': current_year,
                    'round': current_round,
                    'session': session,
                    'event_name': session.event['EventName'] if hasattr(session, 'event') and session.event is not None else f"Round {current_round}"
                }
                
                if is_wet and len(wet_races) < num_wet:
                    wet_races.append(race_data)
                    print(f"  -> Added as WET race ({len(wet_races)}/{num_wet})")
                elif not is_wet and len(dry_races) < num_dry:
                    dry_races.append(race_data)
                    print(f"  -> Added as DRY race ({len(dry_races)}/{num_dry})")
                else:
                    print(f"  -> Skipping race (already have enough {'wet' if is_wet else 'dry'} races)")
                
                current_round -= 1
                
            except Exception as e:
                print(f"Error loading {current_year} Round {current_round}: {e}")
                current_round -= 1
                continue
        
        print(f"\nCompleted search. Found {len(dry_races)} dry races and {len(wet_races)} wet races")
        return dry_races, wet_races
    
    def get_comprehensive_driver_team_mapping(self, target_year, target_round):
        """
        Get comprehensive driver-to-team mapping by checking multiple sessions
        to catch mid-season entries and team changes
        """
        driver_team_map = {}
        
        # Check multiple rounds to catch all drivers (including mid-season entries)
        rounds_to_check = []
        
        # Add target round and several rounds before it (pero asegurándonos de que existan)
        for r in range(max(1, target_round - 3), target_round + 1):
            # Validar cada ronda antes de agregarla
            is_valid, _, _ = self.validate_round(target_year, r)
            if is_valid:
                rounds_to_check.append(r)
        
        # Also check some random rounds from earlier in the season
        if target_round > 5:
            for r in [1, target_round // 2]:
                is_valid, _, _ = self.validate_round(target_year, r)
                if is_valid and r not in rounds_to_check:
                    rounds_to_check.append(r)
        
        print(f"Checking rounds for driver mapping: {rounds_to_check}")
        
        for round_num in rounds_to_check:
            try:
                # Try multiple session types to maximize coverage
                for session_type in ['Race']:
                    try:
                        session = get_session_data(target_year, round_num, session_type)
                        
                        if session is None:
                            continue
                            
                        if not hasattr(session, 'results') or session.results is None or session.results.empty:
                            continue
                        
                        for _, driver in session.results.iterrows():
                            driver_abbr = driver['Abbreviation']
                            team_name = driver['TeamName']
                            
                            # Always update with the most recent team info
                            driver_team_map[driver_abbr] = team_name
                        
                        print(f"  -> Found {len(session.results)} drivers in {target_year} Round {round_num}")
                        break  # If we got data from this round, move to next round
                    except Exception as e:
                        print(f"  -> Error loading {target_year} Round {round_num} {session_type}: {e}")
                        continue
                        
            except Exception as e:
                print(f"Warning: Could not load round {round_num}: {e}")
                continue
        
        print(f"Found {len(driver_team_map)} drivers in team mapping")
        return driver_team_map

    def get_team_average_performance(self, driver_performance, target_year, target_round):
        """
        Calculate team-based average performance for drivers with insufficient data
        """
        try:
            # Get comprehensive driver-team mapping
            driver_team_map = self.get_comprehensive_driver_team_mapping(target_year, target_round)
            
            # Calculate team averages
            team_dry_scores = {}
            team_wet_scores = {}
            
            for driver, scores in driver_performance.items():
                if driver in driver_team_map:
                    team = driver_team_map[driver]
                    
                    if team not in team_dry_scores:
                        team_dry_scores[team] = []
                        team_wet_scores[team] = []
                    
                    team_dry_scores[team].extend(scores['dry_scores'])
                    team_wet_scores[team].extend(scores['wet_scores'])
                else:
                    # Handle drivers not found in current season mapping
                    print(f"Warning: Driver {driver} not found in current season mapping")
            
            # Calculate averages
            team_averages = {}
            for team in team_dry_scores:
                team_averages[team] = {
                    'dry_avg': np.mean(team_dry_scores[team]) if team_dry_scores[team] else 0.0,
                    'wet_avg': np.mean(team_wet_scores[team]) if team_wet_scores[team] else 0.0
                }
            
            return driver_team_map, team_averages
            
        except Exception as e:
            print(f"Error calculating team averages: {e}")
            return {}, {}

    def calculate_performance_factors(self, target_year, target_round, min_dry_races=3, min_wet_races=2):
        """
        Calculate dry and wet performance factors for all drivers
        Handles rookies and drivers with insufficient data
        
        Parameters:
        - min_dry_races: Minimum races required for reliable dry factor (default: 3)
        - min_wet_races: Minimum races required for reliable wet factor (default: 2)
        """
        print(f"Calculating performance factors for {target_year} Round {target_round}")
        print("=" * 60)
        
        # Validate target round first
        is_valid, max_rounds, error_msg = self.validate_round(target_year, target_round)
        if not is_valid:
            raise ValueError(f"❌ Error al cargar los datos: {error_msg}")
        
        print(f"✅ Round {target_round} is valid for {target_year} (max: {max_rounds})")
        
        # Get previous races
        dry_races, wet_races = self.get_previous_races(target_year, target_round)
        
        print(f"\nFound {len(dry_races)} dry races and {len(wet_races)} wet races")
        
        # Get current season drivers for reference (check multiple rounds for completeness)
        current_drivers = set()
        try:
            # Check multiple rounds to catch mid-season entries
            rounds_to_check = [target_round]
            if target_round > 1:
                rounds_to_check.append(target_round - 1)
            if target_round > 5:
                rounds_to_check.append(target_round // 2)
            
            for round_num in rounds_to_check:
                try:
                    # Validate round before loading
                    is_valid, _, _ = self.validate_round(target_year, round_num)
                    if not is_valid:
                        continue
                        
                    for session_type in ['Qualifying']:
                        try:
                            session = get_session_data(target_year, round_num, session_type)
                            if not session.results.empty:
                                current_drivers.update(session.results['Abbreviation'].unique())
                                break
                        except:
                            continue
                except:
                    continue
                    
        except Exception as e:
            print(f"Warning: Could not load current season drivers: {e}")
            current_drivers = set()
        
        # Calculate performance factors
        driver_performance = {}
        
        # Process dry races
        for race in dry_races:
            session = race['session']
            results = session.results
            
            for _, driver in results.iterrows():
                driver_abbr = driver['Abbreviation']
                if driver_abbr not in driver_performance:
                    driver_performance[driver_abbr] = {'dry_scores': [], 'wet_scores': []}
                
                score = self.calculate_race_performance_score(
                    results[results['Abbreviation'] == driver_abbr]
                )
                driver_performance[driver_abbr]['dry_scores'].append(score)
        
        # Process wet races
        for race in wet_races:
            session = race['session']
            results = session.results
            
            for _, driver in results.iterrows():
                driver_abbr = driver['Abbreviation']
                if driver_abbr not in driver_performance:
                    driver_performance[driver_abbr] = {'dry_scores': [], 'wet_scores': []}
                
                score = self.calculate_race_performance_score(
                    results[results['Abbreviation'] == driver_abbr]
                )
                driver_performance[driver_abbr]['wet_scores'].append(score)
        
        # Get team averages for drivers with insufficient data
        driver_team_map, team_averages = self.get_team_average_performance(driver_performance, target_year, target_round)
        
        # Add current drivers who might not be in historical data (rookies/mid-season entries)
        for driver in current_drivers:
            if driver not in driver_performance:
                driver_performance[driver] = {'dry_scores': [], 'wet_scores': []}
        
        # Also add any drivers from team mapping who might not be in current_drivers
        # (handles edge cases where driver_team_map has more complete data)
        for driver in driver_team_map:
            if driver not in driver_performance:
                driver_performance[driver] = {'dry_scores': [], 'wet_scores': []}
        
        # Calculate final performance factors
        performance_factors = []
        
        for driver, scores in driver_performance.items():
            dry_scores = scores['dry_scores']
            wet_scores = scores['wet_scores']
            
            # Check if driver has sufficient data
            has_enough_dry = len(dry_scores) >= min_dry_races
            has_enough_wet = len(wet_scores) >= min_wet_races
            
            # Calculate performance factors
            if has_enough_dry:
                dry_factor = np.mean(dry_scores)
                dry_confidence = 'High'
            elif dry_scores:  # Some data but not enough
                dry_factor = np.mean(dry_scores)
                dry_confidence = 'Low'
            else:  # No data - use team average or default
                if driver in driver_team_map and driver_team_map[driver] in team_averages:
                    dry_factor = team_averages[driver_team_map[driver]]['dry_avg']
                    dry_confidence = 'Team_Avg'
                else:
                    dry_factor = 0.0  # Neutral default for complete rookies
                    dry_confidence = 'Default'
            
            if has_enough_wet:
                wet_factor = np.mean(wet_scores)
                wet_confidence = 'High'
            elif wet_scores:  # Some data but not enough
                wet_factor = np.mean(wet_scores)
                wet_confidence = 'Low'
            else:  # No data - use team average or default
                if driver in driver_team_map and driver_team_map[driver] in team_averages:
                    wet_factor = team_averages[driver_team_map[driver]]['wet_avg']
                    wet_confidence = 'Team_Avg'
                else:
                    wet_factor = 0.0  # Neutral default for complete rookies
                    wet_confidence = 'Default'
            
            # Normalize factors (optional: scale to 0-100 range)
            dry_factor_normalized = max(0, min(100, (dry_factor + 10) * 5))
            wet_factor_normalized = max(0, min(100, (wet_factor + 10) * 5))
            
            # Determine if this is likely a rookie
            is_rookie = len(dry_scores) == 0 and len(wet_scores) == 0
            
            performance_factors.append({
                'Driver': driver,
                'dry_performance_factor': round(dry_factor, 3),
                'wet_performance_factor': round(wet_factor, 3),
                'dry_performance_factor_normalized': round(dry_factor_normalized, 1),
                'wet_performance_factor_normalized': round(wet_factor_normalized, 1),
                'dry_races_count': len(dry_scores),
                'wet_races_count': len(wet_scores),
                'dry_confidence': dry_confidence,
                'wet_confidence': wet_confidence,
                'is_rookie': is_rookie,
                'team': driver_team_map.get(driver, 'Unknown')
            })
        
        # Convert to DataFrame and sort by driver name
        df = pd.DataFrame(performance_factors)
        df = df.sort_values('Driver').reset_index(drop=True)
        
        return df, dry_races, wet_races
    
    def _display_validation_error(self, error_msg, max_rounds, target_year):
        """Helper method to display validation errors with suggestions"""
        print(f"❌ {error_msg}")
        print(f"💡 Sugerencia: Usa una ronda entre 1 y {max_rounds} para el año {target_year}")
    
    def _suggest_available_rounds(self, target_year):
        """Helper method to suggest available rounds when there's an error"""
        try:
            schedule = fastf1.get_event_schedule(target_year)
            print(f"\n💡 Rondas disponibles para {target_year}:")
            for i, event in schedule.iterrows():
                print(f"  Round {i+1}: {event['EventName']} - {event['Location']}")
        except:
            print(f"💡 Prueba con rondas entre 1 y 24 para el año {target_year}")
    
    def _calculate_and_display_results(self, calculator, target_year, target_round, location):
        """Helper method to calculate and display performance factors"""
        performance_df, dry_races, wet_races = calculator.calculate_performance_factors(
            target_year, target_round
        )
        
        print("\n" + "=" * 80)
        print("PERFORMANCE FACTORS SUMMARY")
        print("=" * 80)
        
        print(f"\nBased on analysis of:")
        print(f"DRY RACES ({len(dry_races)}):")
        for race in dry_races:
            print(f"  - {race['year']} Round {race['round']}: {race['event_name']}")
        
        print(f"\nWET RACES ({len(wet_races)}):")
        for race in wet_races:
            print(f"  - {race['year']} Round {race['round']}: {race['event_name']}")
        
        self._display_performance_table(performance_df)
        self._display_summary_statistics(performance_df)
        self._save_results(performance_df, target_year, target_round, location)
    
    def _display_performance_table(self, performance_df):
        """Helper method to display the performance factors table"""
        print(f"\nPERFORMANCE FACTORS:")
        print("=" * 120)
        
        display_df = performance_df.copy()
        
        # Add confidence indicators to the display
        display_df['Dry_Factor_Display'] = display_df.apply(
            lambda x: f"{x['dry_performance_factor']:.3f} ({x['dry_confidence']})", axis=1
        )
        display_df['Wet_Factor_Display'] = display_df.apply(
            lambda x: f"{x['wet_performance_factor']:.3f} ({x['wet_confidence']})", axis=1
        )
        
        # Select columns for display
        display_columns = ['Driver', 'team', 'Dry_Factor_Display', 'Wet_Factor_Display', 
                          'dry_races_count', 'wet_races_count', 'is_rookie']
        
        print(display_df[display_columns].to_string(index=False))
    
    def _display_summary_statistics(self, performance_df):
        """Helper method to display summary statistics"""
        rookies = performance_df[performance_df['is_rookie'] == True]
        experienced = performance_df[performance_df['is_rookie'] == False]
        
        print(f"\n" + "=" * 60)
        print("SUMMARY STATISTICS:")
        print(f"Total Drivers: {len(performance_df)}")
        print(f"Rookies (no historical data): {len(rookies)}")
        print(f"Experienced Drivers: {len(experienced)}")
        
        if len(rookies) > 0:
            print(f"\nROOKIE DRIVERS (using team averages or defaults):")
            for _, rookie in rookies.iterrows():
                print(f"  - {rookie['Driver']} ({rookie['team']})")
        
        # Show confidence level breakdown
        dry_confidence_counts = performance_df['dry_confidence'].value_counts()
        wet_confidence_counts = performance_df['wet_confidence'].value_counts()
        
        print(f"\nCONFIDENCE LEVELS:")
        print(f"Dry Performance Confidence: {dict(dry_confidence_counts)}")
        print(f"Wet Performance Confidence: {dict(wet_confidence_counts)}")
    
    def _save_results(self, performance_df, target_year, target_round, location):
        """Helper method to save results to CSV"""
        filename = f"../data/processed/performance/f1_performance_factors_{target_year}_R{target_round}_{location}.csv"
        performance_df.to_csv(filename, index=False)
        print(f"\nResults saved to {filename}")

In [13]:
calculator = F1PerformanceCalculator()

# Adjust these values for your specific race
target_year = 2025
target_round = 9 
location = 'Spain'

def main():
    """Main execution function with improved error handling"""
    # Debug: Imprimir valores antes de la validación
    print(f"🔍 Intentando validar: Año {target_year}, Ronda {target_round}")
    
    try:
        # Early return para validación - Guard clause
        is_valid, max_rounds, error_msg = calculator.validate_round(target_year, target_round)
        print(f"🔍 Resultado de validación: válido={is_valid}, max_rondas={max_rounds}")
        
        if not is_valid:
            calculator._display_validation_error(error_msg, max_rounds, target_year)
            return  # Early return - evita el else gigante
        
        # Si llegamos aquí, la validación pasó
        calculator._calculate_and_display_results(calculator, target_year, target_round, location)
        
    except Exception as e:
        print(f"❌ Error en la ejecución principal: {e}")
        import traceback
        traceback.print_exc()
        
        # Manejo específico para errores de ronda
        if "Invalid round" in str(e) or "round" in str(e).lower():
            calculator._suggest_available_rounds(target_year)

# Ejecutar la función principal
main()

🔍 Intentando validar: Año 2025, Ronda 9


core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


🔍 Resultado de validación: válido=True, max_rondas=25
Calculating performance factors for 2025 Round 9
✅ Round 9 is valid for 2025 (max: 25)
Loading 2025 Round 8...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '16', '81', '1', '44', '6', '31', '30', '23', '55', '63', '87', '43', '5', '18', '27', '22', '12', '14', '10']
core           INFO 	Loading data for Emilia Romagna Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


✓ Datos cargados exitosamente para Race - 8 2025
  Fecha: 2025-05-25 13:00:00
  Circuito: Monaco Grand Prix
  Ubicación: Monaco
  -> Added as DRY race (1/7)
Loading 2025 Round 7...


...

✓ Datos cargados exitosamente para Race - 9 2025
  Fecha: 2025-06-01 13:00:00
  Circuito: Spanish Grand Prix
  Ubicación: Barcelona
  -> Found 19 drivers in 2025 Round 9


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '2

✓ Datos cargados exitosamente para Race - 1 2025
  Fecha: 2025-03-16 04:00:00
  Circuito: Australian Grand Prix
  Ubicación: Melbourne
  -> Found 20 drivers in 2025 Round 1


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '63', '4', '16', '44', '1', '10', '31', '22', '87', '12', '23', '6', '7', '14', '30', '18', '5', '55', '27']


✓ Datos cargados exitosamente para Race - 4 2025
  Fecha: 2025-04-13 15:00:00
  Circuito: Bahrain Grand Prix
  Ubicación: Sakhir
  -> Found 20 drivers in 2025 Round 4
Found 21 drivers in team mapping

PERFORMANCE FACTORS SUMMARY

Based on analysis of:
DRY RACES (7):
  - 2025 Round 8: Monaco Grand Prix
  - 2025 Round 7: Emilia Romagna Grand Prix
  - 2025 Round 6: Miami Grand Prix
  - 2025 Round 5: Saudi Arabian Grand Prix
  - 2025 Round 4: Bahrain Grand Prix
  - 2025 Round 3: Japanese Grand Prix
  - 2025 Round 2: Chinese Grand Prix

WET RACES (4):
  - 2025 Round 1: Australian Grand Prix
  - 2024 Round 21: São Paulo Grand Prix
  - 2024 Round 12: British Grand Prix
  - 2024 Round 9: Canadian Grand Prix

PERFORMANCE FACTORS:
Driver            team Dry_Factor_Display Wet_Factor_Display  dry_races_count  wet_races_count  is_rookie
   ALB        Williams       3.657 (High)      -4.100 (High)                7                4      False
   ALO    Aston Martin      -3.371 (High)      -0.900 (Hi